In [148]:
import numpy as np
import scipy
import pandas as pd
import math
import matplotlib.pyplot as plt
import random
import sklearn
import spacy
nlp = spacy.load('de_core_news_md')
# import fasttext

import re
from bs4 import BeautifulSoup
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

In [149]:
articles = pd.read_csv('./data/articles_clean.csv')

In [150]:
articles.head()

,nzz_id,author,catchline,content,department,lead_text,pub_date,title,content_len
0,ld.149648,Claudia Gabriel,Obligationenfonds mit fixer Laufzeit,obligationenfonds mit fixer laufzeit es gi...,Finanzen,Die Idee ist gut: Statt einer einzigen Obligat...,2017-03-09 08:01:21.000,Es gibt noch interessante Varianten,718
1,1.18145900,Unknown,Unknown,,Panorama,Zum Auftakt der Fashion Week in New York zeige...,2017-04-11 14:00:29.473,Fashion Week New York,0
2,ld.138769,Unknown,E-Banking-Ausfall,e banking ausfall postfinance kämpft mit d...,Finanzen,Seit Sonntag funktioniert das E-Banking der Po...,2017-01-09 13:55:00.000,Postfinance kämpft mit dem System,525
3,ld.143700,Unknown,Terror in Frankreich,terror in frankreich louvre nach macheten ...,International,Einen Tag nach dem Angriff auf Soldaten beim P...,2017-02-04 12:50:25.000,Louvre nach Macheten-Angriff wieder geöffnet,181
4,ld.149385,Unknown,Unglück in Panama,unglück in panama bus prallt gegen eine ma...,Panorama,Bei einem Busunglück in Panama sind 17 Persone...,2017-03-06 07:31:21.000,Bus prallt gegen eine Mauer und stürzt in Fluss,110


### Lematyzacja

In [111]:
def lemmatizer(data): 
    doc = nlp(data)
    result = ' '.join([x.lemma_ for x in doc]) 
    return result

In [117]:
%%time
content_lemmatized = articles['content'].apply(lemmatizer)

Wall time: 40min 48s


In [121]:
pd.DataFrame(content_lemmatized).head()

,content
0,obligationenfonds mit fixer laufzeit ich...
1,
2,e banking ausfall postfinance kämpfen mi...
3,terror in frankreich louvre nach machete...
4,unglück in panama bus prallen gegen eine...


In [123]:
articles_lemma = articles.copy()

In [127]:
articles_lemma['content'] = content_lemmatized

In [128]:
articles_lemma.head()

,nzz_id,author,catchline,content,department,lead_text,pub_date,title,content_len
0,ld.149648,Claudia Gabriel,Obligationenfonds mit fixer Laufzeit,obligationenfonds mit fixer laufzeit ich...,Finanzen,Die Idee ist gut: Statt einer einzigen Obligat...,2017-03-09 08:01:21.000,Es gibt noch interessante Varianten,718
1,1.18145900,Unknown,Unknown,,Panorama,Zum Auftakt der Fashion Week in New York zeige...,2017-04-11 14:00:29.473,Fashion Week New York,0
2,ld.138769,Unknown,E-Banking-Ausfall,e banking ausfall postfinance kämpfen mi...,Finanzen,Seit Sonntag funktioniert das E-Banking der Po...,2017-01-09 13:55:00.000,Postfinance kämpft mit dem System,525
3,ld.143700,Unknown,Terror in Frankreich,terror in frankreich louvre nach machete...,International,Einen Tag nach dem Angriff auf Soldaten beim P...,2017-02-04 12:50:25.000,Louvre nach Macheten-Angriff wieder geöffnet,181
4,ld.149385,Unknown,Unglück in Panama,unglück in panama bus prallen gegen eine...,Panorama,Bei einem Busunglück in Panama sind 17 Persone...,2017-03-06 07:31:21.000,Bus prallt gegen eine Mauer und stürzt in Fluss,110


In [146]:
articles_lemma.to_csv('./data/articles_lemmatized.csv', index=False, encoding='utf-8')